# Carregar dados do Lakehouse Bronze para Silver


In [1]:
# Definir o caminho ou referência da tabela Bronze (tabela "produto" no Lakehouse Bronze)

bronze_table = "Bronze_Lakehouse_GL.produto"  # Nome da tabela no Lakehouse Bronze

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 3, Finished, Available, Finished)

In [2]:
# Carregar os dados da camada Bronze
df_bronze = spark.read.format("delta").table(bronze_table)  # Usar a variável bronze_table como string

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 4, Finished, Available, Finished)

In [3]:
# Exibir o schema para garantir que os dados foram carregados corretamente
df_bronze.printSchema()

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 5, Finished, Available, Finished)

root
 |-- IDSK: string (nullable = true)
 |-- IDPRODUTO: string (nullable = true)
 |-- NOME: string (nullable = true)
 |-- ID_CATEGORIA: string (nullable = true)
 |-- CUSTO_MEDIO: string (nullable = true)
 |-- VALOR_UNITARIO: string (nullable = true)



# Transformações no DataFrame (opcional)

O objetivo da transformação para a camada Silver é:

**Limpeza de Dados:** Eliminar dados inválidos, nulos ou duplicados.

**Estruturação:** Organizar os dados de uma forma mais estruturada, como a separação de campos compostos ou a conversão de tipos de dados.

**Enriquecimento Básico:** Você pode adicionar dados adicionais, como a conversão de códigos em valores legíveis ou a adição de metadados para melhor compreensão dos dados.

**Normalização:** Ajustar os dados para garantir que tenham formatos e unidades consistentes

In [4]:
# Realizar transformações, se necessário
## Exemplo:  df_transformed = df_bronze.select("column1", "column2")  # Exemplo de transformação

from pyspark.sql.functions import col

# Conversão de tipos de dados
df_transformed = df_bronze.withColumn("CUSTO_MEDIO", col("CUSTO_MEDIO").cast("double")) \
                          .withColumn("VALOR_UNITARIO", col("VALOR_UNITARIO").cast("double"))


StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 6, Finished, Available, Finished)

In [5]:
from pyspark.sql.functions import col, upper, expr

# Tratar valores nulos
df_transformed = df_transformed.fillna({
    "IDSK": "Desconhecido",
    "IDPRODUTO": "Desconhecido",
    "NOME": "Sem Nome",
    "ID_CATEGORIA": "Desconhecida",
    "CUSTO_MEDIO": 0.0,
    "VALOR_UNITARIO": 0.0
})

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 7, Finished, Available, Finished)

In [15]:
from pyspark.sql.functions import col, lower

# Renomear todas as colunas para letras minúsculas
df_transformed = df_transformed.toDF(*[c.lower() for c in df_transformed.columns])

# Converter o conteúdo das colunas de texto para letras minúsculas
colunas_texto = ["nome", "id_categoria"]  # Colunas que possuem texto e devem ser convertidas
for coluna in colunas_texto:
    df_transformed = df_transformed.withColumn(coluna, lower(col(coluna)))

# Mostrar o DataFrame transformado
df_transformed.show()

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 17, Finished, Available, Finished)

+----+---------+--------------------+------------+-----------+--------------+------------+
|idsk|idproduto|                nome|id_categoria|custo_medio|valor_unitario|margem_bruta|
+----+---------+--------------------+------------+-----------+--------------+------------+
|   1|        1|cavaleiro dos 7 r...|           1|       15.0|          30.0|        15.0|
|  10|       10|         tsql tuning|           1|       75.0|          90.0|        15.0|
| 100|      100|pendrive 128 cros...|           5|       98.0|         137.0|        39.0|
| 101|      101|        pendrive 128|           5|      100.0|         146.0|        46.0|
| 102|      102|       hd 1t samsung|           5|      110.0|         260.0|       150.0|
| 103|      103|       hd 2t samsung|           5|      200.0|         550.0|       350.0|
| 104|      104|       hd 3t samsung|           5|      300.0|         700.0|       400.0|
| 105|      105|               hd 1t|           5|      150.0|         290.0|       140.0|

In [7]:
from pyspark.sql.functions import col, upper, expr


# Criar coluna derivada: Margem Bruta
df_transformed = df_transformed.withColumn("MARGEM_BRUTA", expr("VALOR_UNITARIO - CUSTO_MEDIO"))

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 9, Finished, Available, Finished)

In [8]:
from pyspark.sql.functions import col, upper, expr


# Remover duplicatas
df_transformed = df_transformed.dropDuplicates(["IDPRODUTO", "IDSK"])


StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 10, Finished, Available, Finished)

In [9]:
from pyspark.sql.functions import col, upper, expr

# Validar valores numéricos
df_transformed = df_transformed.filter((col("CUSTO_MEDIO") >= 0) & (col("VALOR_UNITARIO") >= 0))

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 11, Finished, Available, Finished)

In [10]:
# Padronize os nomes das colunas para letras minúsculas
df_transformed = df_transformed.select([col(col_name).alias(col_name.lower()) for col_name in df_transformed.columns])

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 12, Finished, Available, Finished)

In [16]:
from pyspark.sql.functions import col, upper, expr

# Exibir esquema e algumas linhas para verificar
df_transformed.printSchema()
df_transformed.show(5)

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 18, Finished, Available, Finished)

root
 |-- idsk: string (nullable = false)
 |-- idproduto: string (nullable = false)
 |-- nome: string (nullable = false)
 |-- id_categoria: string (nullable = false)
 |-- custo_medio: double (nullable = false)
 |-- valor_unitario: double (nullable = false)
 |-- margem_bruta: double (nullable = false)

+----+---------+--------------------+------------+-----------+--------------+------------+
|idsk|idproduto|                nome|id_categoria|custo_medio|valor_unitario|margem_bruta|
+----+---------+--------------------+------------+-----------+--------------+------------+
|   1|        1|cavaleiro dos 7 r...|           1|       15.0|          30.0|        15.0|
|  10|       10|         tsql tuning|           1|       75.0|          90.0|        15.0|
| 100|      100|pendrive 128 cros...|           5|       98.0|         137.0|        39.0|
| 101|      101|        pendrive 128|           5|      100.0|         146.0|        46.0|
| 102|      102|       hd 1t samsung|           5|      110.

# Salvar no Lakehouse Silver

In [17]:
# Definir o caminho ou referência da tabela Silver
silver_table = "Silver_Lakehouse_GL.produto"  # Nome da tabela no Lakehouse Silver

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 19, Finished, Available, Finished)

In [18]:
# Gravar os dados na camada Silver
df_transformed.write.format("delta").mode("overwrite").saveAsTable(silver_table)  # Salva a tabela no formato Delta

StatementMeta(, f77a3984-1747-4ffa-b114-56beafc90de8, 20, Finished, Available, Finished)